# Une classe pour simplifier l'accès aux données

In [28]:
object PrenomUtils extends Serializable {
    def extractField(s: String, fieldNumber: Int): String = {
        val fields = s.split('\t')
        if (fieldNumber >= fields.length) "" else fields(fieldNumber)
    }
}

In [29]:
println(PrenomUtils.extractField("2	CASSIOPEE	2009	33	3.0000", 0))
println(PrenomUtils.extractField("2	CASSIOPEE	2009	33	3.0000", 1))
println(PrenomUtils.extractField("2	CASSIOPEE	2009	33	3.0000", 2))
println(PrenomUtils.extractField("2	CASSIOPEE	2009	33	3.0000", 3))
println(PrenomUtils.extractField("2	CASSIOPEE	2009	33	3.0000", 4))
println(PrenomUtils.extractField("2	CASSIOPEE	2009	33	3.0000", 5))

2
CASSIOPEE
2009
33
3.0000


In [38]:
case class Prenom(
        val sexe: Char, // 1 pour masculin, 2 pour féminin
        val prenom: String,
        val annee: Int, // 1900 à 2015, XXXX
        val dept: Int, // liste des départements, XX
        val nombre: Double
    ) {
    def this(s: String) = this(
        PrenomUtils.extractField(s, 0).charAt(0),
        PrenomUtils.extractField(s, 1),
        PrenomUtils.extractField(s, 2).toInt,
        PrenomUtils.extractField(s, 3).toInt,
        PrenomUtils.extractField(s, 4).toDouble
    )
}

In [39]:
val ariane = new Prenom('2', "ARIANE", 2007, 91, 3.0)
println(ariane)
val cassiopee = new Prenom("2	CASSIOPEE	2009	33	3.0000")
println(cassiopee)
val helios = Prenom('1', "HELIOS", 2010, 59, 3.0)
println(helios)

Prenom(2,ARIANE,2007,91,3.0)
Prenom(2,CASSIOPEE,2009,33,3.0)
Prenom(1,HELIOS,2010,59,3.0)


# Charger les données
1. Créer le RDD `lignes` à partir du répertoire `prenoms_sample.txt`

In [48]:
val lignes = sc.textFile("prenoms_sample.txt")
lignes.take(10).foreach(println)

1	AARON	2011	17	13.0000
1	AARON	2015	33	49.0000
1	ABDALLAH	1977	42	4.0000
1	ABDALLAH	2005	92	13.0000
1	ABDELAZIZ	1971	28	3.0000
1	ABDELAZIZ	1978	08	3.0000
1	ABDERRAHMAN	2011	06	3.0000
1	ABDOULAYE	2005	75	23.0000
1	ABEL	1908	58	8.0000
1	ADAM	2002	22	3.0000


# Transformer les lignes en prénoms
1. En appliquant la méthode `map`, créer le RDD prenoms à partir de `lignes`

In [50]:
val prenoms = lignes.map(l => (
    PrenomUtils.extractField(l, 0).charAt(0),
    PrenomUtils.extractField(l, 1),
    PrenomUtils.extractField(l, 2).toInt,
    PrenomUtils.extractField(l, 3).toInt,
    PrenomUtils.extractField(l, 4).toDouble
))
prenoms.take(10).foreach(println)

(1,AARON,2011,17,13.0)
(1,AARON,2015,33,49.0)
(1,ABDALLAH,1977,42,4.0)
(1,ABDALLAH,2005,92,13.0)
(1,ABDELAZIZ,1971,28,3.0)
(1,ABDELAZIZ,1978,8,3.0)
(1,ABDERRAHMAN,2011,6,3.0)
(1,ABDOULAYE,2005,75,23.0)
(1,ABEL,1908,58,8.0)
(1,ADAM,2002,22,3.0)
